In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df=pd.read_csv("heart_clean_final.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,62,0,0,138,294.0,1,1,106,0,1.9,1,3,2,0
1,58,0,0,100,248.0,0,0,122,0,1.0,1,0,2,1
2,58,1,0,114,318.0,0,2,140,0,3.5,0,3,1,0
3,54,1,0,122,286.0,0,0,116,1,3.2,1,2,2,0
4,71,0,0,112,149.0,0,1,125,0,1.6,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,60,0,2,120,178.0,1,1,100,0,0.0,2,0,2,1
179,64,1,2,140,335.0,0,1,158,0,0.0,2,0,2,0
180,68,0,2,120,211.0,0,0,115,0,1.5,1,0,2,1
181,44,0,2,108,141.0,0,1,175,0,0.6,1,0,2,1


In [5]:
indep=df.drop("target",axis=1)
dep=df["target"]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [113]:
param_grid = {'max_depth':[5,10,20,30],'learning_rate':[0.01,0.1,0.5,1.0],'subsample':[0.1,0.5,0.7,1.0],'colsample_bytree':[0.1,0.5,0.7,1.0],"gamma":[0,1,2]} 
grid = GridSearchCV(XGBClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 768 candidates, totalling 3840 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.1, 0.5, 0.7, 1.0],
                         'gamma': [0, 1, 2],
                         'learning_rate': [0.01, 0.1, 0.5, 1.0],
                         'max_depth': [5, 10, 20, 30],
                         'subsample': [0.1, 0.5, 0.7, 1.0]},
             scoring='f1_weighted', verbose=3)

In [115]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [117]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'colsample_bytree': 0.5, 'gamma': 2, 'learning_rate': 0.5, 'max_depth': 5, 'subsample': 1.0}: 0.822886531661845


In [119]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[ 5  7]
 [ 3 46]]


In [121]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.62      0.42      0.50        12
           1       0.87      0.94      0.90        49

    accuracy                           0.84        61
   macro avg       0.75      0.68      0.70        61
weighted avg       0.82      0.84      0.82        61



In [123]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.818877551020408

In [125]:
table=pd.DataFrame.from_dict(re)

In [127]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018497,0.004101,0.008923,0.002364,0.1,0,0.01,5,0.1,"{'colsample_bytree': 0.1, 'gamma': 0, 'learnin...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,685
1,0.021916,0.001265,0.009911,0.002154,0.1,0,0.01,5,0.5,"{'colsample_bytree': 0.1, 'gamma': 0, 'learnin...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,685
2,0.020674,0.002722,0.009108,0.001531,0.1,0,0.01,5,0.7,"{'colsample_bytree': 0.1, 'gamma': 0, 'learnin...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,685
3,0.019322,0.003621,0.006713,0.000238,0.1,0,0.01,5,1.0,"{'colsample_bytree': 0.1, 'gamma': 0, 'learnin...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,685
4,0.018735,0.001333,0.008244,0.001154,0.1,0,0.01,10,0.1,"{'colsample_bytree': 0.1, 'gamma': 0, 'learnin...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0.016316,0.000678,0.006352,0.001164,1.0,2,1.00,20,1.0,"{'colsample_bytree': 1.0, 'gamma': 2, 'learnin...",0.785031,0.840000,0.786630,0.786630,0.771704,0.793999,0.023674,301
764,0.018016,0.001451,0.006506,0.001645,1.0,2,1.00,30,0.1,"{'colsample_bytree': 1.0, 'gamma': 2, 'learnin...",0.767366,0.699044,0.701282,0.615934,0.808772,0.718480,0.065927,661
765,0.018319,0.002565,0.006301,0.000981,1.0,2,1.00,30,0.5,"{'colsample_bytree': 1.0, 'gamma': 2, 'learnin...",0.749524,0.680000,0.877273,0.701282,0.771704,0.755956,0.068943,552
766,0.016618,0.001775,0.005906,0.000583,1.0,2,1.00,30,0.7,"{'colsample_bytree': 1.0, 'gamma': 2, 'learnin...",0.833016,0.840000,0.871978,0.750000,0.863022,0.831603,0.043241,33
